In [2]:
import pandas as pd
file_name = 'Hualien_tripadvisor_top1500'
df = pd.read_csv('../0815/'+file_name+'.csv')

In [3]:
count = 0
for i, value in df['uri'].value_counts().iteritems():
    if value > 1:
        count +=1
print(count)

190


### 顯示重複

In [4]:
df.loc[df['uri'] == df['uri'].value_counts().index[0]]

,title,paiming,recommend_price,uri,hotel_address,price_range,avg_rating,comment_count,offical_img_uri,facility,room,agoda_price
267,Small Travel Stay,"花蓮排名第 268 的超值住宿地點，共 2,231 家","NT$1,247",https://www.tripadvisor.com.tw/Hotel_Review-g1...,"No.13-1, Xuanyuan Rd.","NT$1,285 - NT$1,818",5.0,1.0,https://media-cdn.tripadvisor.com/media/photo-...,NaN,NaN,NaN
418,Small Travel Stay,"花蓮排名第 419 的超值住宿地點，共 2,231 家",NaN,https://www.tripadvisor.com.tw/Hotel_Review-g1...,"No.13-1, Xuanyuan Rd.","NT$1,285 - NT$1,818",5.0,1.0,https://media-cdn.tripadvisor.com/media/photo-...,NaN,NaN,NaN
486,Small Travel Stay,"花蓮排名第 487 的超值住宿地點，共 2,231 家",NT$952,https://www.tripadvisor.com.tw/Hotel_Review-g1...,"No.13-1, Xuanyuan Rd.","NT$1,285 - NT$1,818",5.0,1.0,https://media-cdn.tripadvisor.com/media/photo-...,NaN,NaN,NT$952
1129,Small Travel Stay,"花蓮排名第 1,130 的超值住宿地點，共 2,231 家",NaN,https://www.tripadvisor.com.tw/Hotel_Review-g1...,"No.13-1, Xuanyuan Rd.","NT$1,285 - NT$1,818",5.0,1.0,https://media-cdn.tripadvisor.com/media/photo-...,NaN,NaN,NaN


### 只保留相同飯店名稱/ uri的第一筆

In [5]:
rows_list = []
# 把row存到list, 之後再轉df
for i, value in df['uri'].value_counts().iteritems():
    # 轉成dict再存到 list
    rows_list.append(df.loc[df['uri'] == i].head(1).to_dict('records')[0])
    
new_df = pd.DataFrame(rows_list)

確定沒有重複的飯店

In [6]:
new_df['uri'].value_counts()[new_df['uri'].value_counts() > 1].index

Index([], dtype='object')

---
#### 移除用不到的特徵

In [7]:
new_df.drop(['paiming', 'offical_img_uri'], axis=1, inplace=True)

空的rating跟comment補0, 表示沒有人評價過

In [8]:
new_df['avg_rating'].fillna(0, inplace=True)
new_df['comment_count'].fillna(0, inplace=True)

---
#### drop掉空值過多的欄位

In [9]:
new_df.isna().sum()/ len(df)

title              0.000000
recommend_price    0.441880
uri                0.000000
hotel_address      0.000000
price_range        0.099145
avg_rating         0.000000
comment_count      0.000000
facility           0.445299
room               0.583761
agoda_price        0.652991
dtype: float64

只有10筆是完整沒有空值

In [10]:
new_df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 28 to 945
Data columns (total 10 columns):
title              48 non-null object
recommend_price    48 non-null object
uri                48 non-null object
hotel_address      48 non-null object
price_range        48 non-null object
avg_rating         48 non-null float64
comment_count      48 non-null float64
facility           48 non-null object
room               48 non-null object
agoda_price        48 non-null object
dtypes: float64(2), object(8)
memory usage: 4.1+ KB


In [11]:
new_df.dropna(axis=1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 5 columns):
title            948 non-null object
uri              948 non-null object
hotel_address    948 non-null object
avg_rating       948 non-null float64
comment_count    948 non-null float64
dtypes: float64(2), object(3)
memory usage: 37.2+ KB


暫時剔除以下的feature

In [12]:
new_df.drop(['recommend_price', 'facility','room', 'agoda_price'], axis=1, inplace=True)

再踢出row有空值的

In [13]:
new_df.dropna(inplace=True)
print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 832 entries, 0 to 947
Data columns (total 6 columns):
title            832 non-null object
uri              832 non-null object
hotel_address    832 non-null object
price_range      832 non-null object
avg_rating       832 non-null float64
comment_count    832 non-null float64
dtypes: float64(2), object(4)
memory usage: 45.5+ KB
None


### 存成新的表

In [14]:
new_df.to_csv('./'+file_name+'_clean.csv', index=False, encoding='utf_8_sig')